### Imports

In [22]:
import os
from os import path
from os.path import join, relpath
import random

import xml.etree.ElementTree as ET

from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

### Helpful constants

In [5]:
colors = 'red','green','purple'
nums = 'one','two','three'
shades = 'empty','partial','full'
shapes = 'diamond','oval','squiggle'
label_names = [f'{num} {color} {shape} {shade}' for num in nums for color in colors for shape in shapes for shade in shades]

### Data formatting

In [3]:
def get_set(path):
    data = pd.DataFrame(columns=['filepath', 'class_color', 'class_num', 'class_shape', 'class_shade'])
    for dirpath, subdirs, files in os.walk(path):
        for file in files:
            filepath = relpath(join(dirpath, file))
            strpath = str(filepath).split('\\')
            class_num = strpath[-5]
            class_color = strpath[-4]
            class_shape = strpath[-3]
            class_shade = strpath[-2]
            row = pd.DataFrame({'filepath': filepath,
                             'class_color': class_color,
                             'class_num': class_num,
                             'class_shape': class_shape,
                             'class_shade': class_shade},
                               columns=['filepath', 'class_color', 'class_num', 'class_shape', 'class_shade'],
                               index=[0])
            data = pd.concat([data, row], ignore_index=True)
    return data

data_panda = get_set('./dataSET')
display(data_panda.head(50))

,filepath,class_color,class_num,class_shape,class_shade
0,dataSET\one\green\diamond\empty\101.png,green,one,diamond,empty
1,dataSET\one\green\diamond\empty\120.png,green,one,diamond,empty
2,dataSET\one\green\diamond\empty\172.png,green,one,diamond,empty
3,dataSET\one\green\diamond\empty\189.png,green,one,diamond,empty
4,dataSET\one\green\diamond\empty\2.png,green,one,diamond,empty
5,dataSET\one\green\diamond\empty\205.png,green,one,diamond,empty
6,dataSET\one\green\diamond\empty\222.png,green,one,diamond,empty
7,dataSET\one\green\diamond\empty\254.png,green,one,diamond,empty
8,dataSET\one\green\diamond\empty\286.png,green,one,diamond,empty
9,dataSET\one\green\diamond\empty\31.png,green,one,diamond,empty


In [5]:
data = get_set('./dataSET')

mlb = MultiLabelBinarizer(classes=colors + nums + shades + shapes)
labels = mlb.fit_transform(data[data.columns[1:]].values)
label_frame = pd.DataFrame({key : row for key, row in zip(mlb.classes, labels.T)})
binarized_data = pd.concat((data['filepath'], label_frame), axis=1)

display(binarized_data.head(50))
binarized_data.to_csv('labels.csv', index=False)

,filepath,red,green,purple,one,two,three,empty,partial,full,diamond,oval,squiggle
0,dataSET\one\green\diamond\empty\101.png,0,1,0,1,0,0,1,0,0,1,0,0
1,dataSET\one\green\diamond\empty\120.png,0,1,0,1,0,0,1,0,0,1,0,0
2,dataSET\one\green\diamond\empty\172.png,0,1,0,1,0,0,1,0,0,1,0,0
3,dataSET\one\green\diamond\empty\189.png,0,1,0,1,0,0,1,0,0,1,0,0
4,dataSET\one\green\diamond\empty\2.png,0,1,0,1,0,0,1,0,0,1,0,0
5,dataSET\one\green\diamond\empty\205.png,0,1,0,1,0,0,1,0,0,1,0,0
6,dataSET\one\green\diamond\empty\222.png,0,1,0,1,0,0,1,0,0,1,0,0
7,dataSET\one\green\diamond\empty\254.png,0,1,0,1,0,0,1,0,0,1,0,0
8,dataSET\one\green\diamond\empty\286.png,0,1,0,1,0,0,1,0,0,1,0,0
9,dataSET\one\green\diamond\empty\31.png,0,1,0,1,0,0,1,0,0,1,0,0


### Image Format
TensorFlow Lite Model Maker requires JPEG images

In [6]:
from PIL import Image
os.makedirs('dataSET.jpg', exist_ok=True)
for dirpath, dirname, filenames in os.walk('dataSET'):
    for file in filenames:
        png = Image.open(join(dirpath, file))
        png.save(join('dataSET.jpg', f'{"_".join(dirpath.split(os.sep)[1:])}_{path.splitext(file)[0]}.jpg'))

### XML data
During label process, png filenames were saved to XML annotations. The code below changes them to the new jpg file extensions.

In [7]:
for dirpath, dirname, filenames in os.walk('boxes'):
    for file in filenames:
        tree = ET.parse(join(dirpath, file))
        root = tree.getroot()
        
        filename_tag = root.find('filename')
        filename_tag.text = f'{"_".join(dirpath.split(os.sep)[1:])}_{path.splitext(file)[0]}.jpg'
        
        folder_tag = root.find('folder')
        folder_tag.text = join(*dirpath.split(os.sep)[1:])
        tree.write(join(dirpath, file))

In [6]:
def split():
    annotations = [relpath(join(dirpath, path.splitext(file)[0]), 'boxes') for (dirpath, dirnames, filenames) in os.walk('boxes') for file in filenames]
    random.shuffle(annotations)
    train_annotations = []
    validation_annotations = []
    test_annotations = []
    for i in range(len(annotations)):
        if i < .8 * len(annotations):
            train_annotations.append(annotations[i])
        elif i < .9 * len(annotations):
            validation_annotations.append(annotations[i])
        else:
            test_annotations.append(annotations[i])

    train_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=train_annotations, label_map=label_names)
    validation_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=validation_annotations, label_map=label_names)
    test_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=test_annotations, label_map=label_names)
    
    return train_data, validation_data, test_data

In [9]:
train_data, val_data, test_data = split()

In [18]:
low_latency_spec = model_spec.get('efficientdet_lite0')
high_precision_spec = model_spec.get('efficientdet_lite4')

In [ ]:
low_latency_model = object_detector.create(train_data, model_spec=low_latency_spec, validation_data=val_data)
low_latency_model.export(tflite_filename='ll_model.tflite', export_dir='.')

In [28]:
high_precision_model = object_detector.create(train_data, model_spec=low_latency_spec, validation_data=val_data)
high_precision_model.export(tflite_filename='hp_model.tflite', export_dir='.')

Epoch 1/50
22/22 [==============================] - 145s 6s/step - det_loss: 2.2874 - cls_loss: 1.7767 - box_loss: 0.0102 - reg_l2_loss: 0.0184 - loss: 2.3059 - learning_rate: 0.0439 - gradient_norm: 0.4848 - val_det_loss: 2.2706 - val_cls_loss: 1.8240 - val_box_loss: 0.0089 - val_reg_l2_loss: 0.0184 - val_loss: 2.2891
Epoch 2/50
22/22 [==============================] - 131s 6s/step - det_loss: 1.9870 - cls_loss: 1.7232 - box_loss: 0.0053 - reg_l2_loss: 0.0185 - loss: 2.0055 - learning_rate: 0.0798 - gradient_norm: 0.5031 - val_det_loss: 2.1309 - val_cls_loss: 1.7691 - val_box_loss: 0.0072 - val_reg_l2_loss: 0.0185 - val_loss: 2.1493
Epoch 3/50
22/22 [==============================] - 132s 6s/step - det_loss: 1.8110 - cls_loss: 1.6508 - box_loss: 0.0032 - reg_l2_loss: 0.0185 - loss: 1.8295 - learning_rate: 0.0795 - gradient_norm: 0.6616 - val_det_loss: 1.9400 - val_cls_loss: 1.6852 - val_box_loss: 0.0051 - val_reg_l2_loss: 0.0185 - val_loss: 1.9586
Epoch 4/50
22/22 [===================

## Results

In [ ]:
label_map = {num: label for num, label in zip(range(1, len(label_names) + 1), label_names)}

In [30]:
ll_obj_detector = object_detector.ObjectDetector(model_spec=low_latency_spec, label_map=test_data.label_map)
ll_obj_detector.evaluate_tflite('ll_model.tflite', test_data)

  3/176 [..............................] - ETA: 12:39

KeyboardInterrupt: 

In [32]:
hh_obj_detector = object_detector.ObjectDetector(model_spec=high_precision_spec, label_map=test_data.label_map)
hh_obj_detector.evaluate_tflite('hp_model.tflite', test_data)

AttributeError: 'PrefetchDataset' object has no attribute 'gen_dataset'

### Running Model

In [23]:
from imageio import imread

In [27]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="ll_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.asarray(imread('dataSET/one/green/diamond/empty/2.png'))
interpreter.set_tensor(input_details[0]['index'], [input_data])

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

ValueError: Cannot set tensor: Dimension mismatch. Got 3 but expected 320 for dimension 1 of input 0.